In [103]:
import numpy as np
import pandas as pd
import scipy
from scipy import stats
from statsmodels.stats.multitest import multipletests
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from statsmodels.stats import diagnostic

In [104]:
data = pd.read_csv('botswana.tsv', sep='\t')

In [105]:
data.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,evermarr,agefm,heduc,urban,electric,radio,tv,bicycle
0,0,18,10,catholic,4.0,1.0,1.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
1,2,43,11,protestant,2.0,1.0,1.0,1,20.0,14.0,1,1.0,1.0,1.0,1.0
2,0,49,4,spirit,4.0,1.0,0.0,1,22.0,1.0,1,1.0,1.0,0.0,0.0
3,0,24,12,other,2.0,1.0,0.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
4,3,32,13,other,3.0,1.0,1.0,1,24.0,12.0,1,1.0,1.0,1.0,1.0


In [106]:
columns = data.columns.values
columns

array(['ceb', 'age', 'educ', 'religion', 'idlnchld', 'knowmeth',
       'usemeth', 'evermarr', 'agefm', 'heduc', 'urban', 'electric',
       'radio', 'tv', 'bicycle'], dtype=object)

In [107]:
len(data['religion'].unique())

4

In [108]:
len(data.dropna())

1834

In [109]:
data['nevermarr'] = pd.DataFrame([0 if item == 1 else 1 for item in data['evermarr']])

In [110]:
data.drop(['evermarr'], axis=1, inplace=True)

In [112]:
data.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,agefm,heduc,urban,electric,radio,tv,bicycle,nevermarr
0,0,18,10,catholic,4.0,1.0,1.0,NaN,NaN,1,1.0,1.0,1.0,1.0,1
1,2,43,11,protestant,2.0,1.0,1.0,20.0,14.0,1,1.0,1.0,1.0,1.0,0
2,0,49,4,spirit,4.0,1.0,0.0,22.0,1.0,1,1.0,1.0,0.0,0.0,0
3,0,24,12,other,2.0,1.0,0.0,NaN,NaN,1,1.0,1.0,1.0,1.0,1
4,3,32,13,other,3.0,1.0,1.0,24.0,12.0,1,1.0,1.0,1.0,1.0,0


In [113]:
data['agefm'].fillna(0, inplace=True)

In [114]:
data.loc[data['nevermarr']==1, 'heduc'] = data.loc[data['nevermarr']==1, 'heduc'].fillna(-1)

In [115]:
data.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,agefm,heduc,urban,electric,radio,tv,bicycle,nevermarr
0,0,18,10,catholic,4.0,1.0,1.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1
1,2,43,11,protestant,2.0,1.0,1.0,20.0,14.0,1,1.0,1.0,1.0,1.0,0
2,0,49,4,spirit,4.0,1.0,0.0,22.0,1.0,1,1.0,1.0,0.0,0.0,0
3,0,24,12,other,2.0,1.0,0.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1
4,3,32,13,other,3.0,1.0,1.0,24.0,12.0,1,1.0,1.0,1.0,1.0,0


In [116]:
data['heduc'].isna().sum()

123

In [117]:
data['idlnchld_noans'] = pd.DataFrame([0 if item == 1 else 1 for item in data['idlnchld']])
data['heduc_noans'] = pd.DataFrame([0 if item == 1 else 1 for item in data['heduc']])
data['usemeth_noans'] = pd.DataFrame([0 if item == 1 else 1 for item in data['usemeth']])

In [118]:
data.idlnchld[data.idlnchld.isnull()] = -1
data.heduc[data.heduc.isnull()] = -2
data.usemeth[data.usemeth.isnull()] = -1

<ipython-input-118-98a91d8cfed7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.idlnchld[data.idlnchld.isnull()] = -1
<ipython-input-118-98a91d8cfed7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.heduc[data.heduc.isnull()] = -2
<ipython-input-118-98a91d8cfed7>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.usemeth[data.usemeth.isnull()] = -1


In [119]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4361 entries, 0 to 4360
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ceb             4361 non-null   int64  
 1   age             4361 non-null   int64  
 2   educ            4361 non-null   int64  
 3   religion        4361 non-null   object 
 4   idlnchld        4361 non-null   float64
 5   knowmeth        4354 non-null   float64
 6   usemeth         4361 non-null   float64
 7   agefm           4361 non-null   float64
 8   heduc           4361 non-null   float64
 9   urban           4361 non-null   int64  
 10  electric        4358 non-null   float64
 11  radio           4359 non-null   float64
 12  tv              4359 non-null   float64
 13  bicycle         4358 non-null   float64
 14  nevermarr       4361 non-null   int64  
 15  idlnchld_noans  4361 non-null   int64  
 16  heduc_noans     4361 non-null   int64  
 17  usemeth_noans   4361 non-null   i

In [120]:
data.dropna(inplace=True)

In [121]:
len(data) * len(data.columns)

78264

In [122]:
data.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,agefm,heduc,urban,electric,radio,tv,bicycle,nevermarr,idlnchld_noans,heduc_noans,usemeth_noans
0,0,18,10,catholic,4.0,1.0,1.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1,1,1,0
1,2,43,11,protestant,2.0,1.0,1.0,20.0,14.0,1,1.0,1.0,1.0,1.0,0,1,1,0
2,0,49,4,spirit,4.0,1.0,0.0,22.0,1.0,1,1.0,1.0,0.0,0.0,0,1,0,1
3,0,24,12,other,2.0,1.0,0.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1,1,1,1
4,3,32,13,other,3.0,1.0,1.0,24.0,12.0,1,1.0,1.0,1.0,1.0,0,1,1,0


In [126]:
m1 = smf.ols('ceb ~ ' + ' + '.join(data.columns[1:]), data=data)
fitted = m1.fit()

In [127]:
fitted.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    ceb   R-squared:                       0.640
Model:                            OLS   Adj. R-squared:                  0.638
Method:                 Least Squares   F-statistic:                     405.1
Date:                Wed, 21 Apr 2021   Prob (F-statistic):               0.00
Time:                        18:18:16   Log-Likelihood:                -7757.4
No. Observations:                4348   AIC:                         1.555e+04
Df Residuals:                    4328   BIC:                         1.568e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -0.2464      0.436     -0.565      0.572      -1.101       0.609
religion[T.other]         -0.0846      0.083     -1.015      0.310      -0.248       0.079
religion[T.protestant]    -0.0132      0.083     -0.159      0.874      -0.176       0.149
religion[T.spirit]        -0.0120      0.078     -0.154      0.878      -0.164       0.140
age                        0.1724      0.003     52.641      0.000       0.166       0.179
educ                      -0.0775      0.007    -10.535      0.000      -0.092      -0.063
idlnchld                   0.0617      0.010      6.154      0.000       0.042       0.081
knowmeth                   0.5240      0.122      4.293      0.000       0.285       0.763
usemeth                   -0.1366      0.183     -0.747      0.455      -0.495       0.222
agefm                     -0.0617      0.007     -9.351      0.000      -0.075      -0.049
heduc                     -0.0394      0.008     -5.215      0.000      -0.054      -0.025
urban                     -0.2226      0.047     -4.692      0.000      -0.316      -0.130
electric                  -0.2928      0.078     -3.775      0.000      -0.445      -0.141
radio                     -0.0283      0.051     -0.551      0.582      -0.129       0.072
tv                        -0.1713      0.093     -1.838      0.066      -0.354       0.011
bicycle                    0.2149      0.051      4.255      0.000       0.116       0.314
nevermarr                 -2.1409      0.147    -14.592      0.000      -2.429      -1.853
idlnchld_noans            -0.3103      0.155     -2.002      0.045      -0.614      -0.006
heduc_noans                0.3031      0.292      1.038      0.300      -0.270       0.876
usemeth_noans             -0.7596      0.197     -3.858      0.000      -1.146      -0.374
==============================================================================
Omnibus:                      224.851   Durbin-Watson:                   1.883
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              859.749
Skew:                           0.023   Prob(JB):                    2.03e-187
Kurtosis:                       5.178   Cond. No.                         736.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Используем критерий Бройша-Пагана для проверки гомоскедастичности ошибок:

In [128]:
print('Breusch-Pagan test: p=%f' % diagnostic.het_breuschpagan(fitted.resid, fitted.model.exog)[1])

Breusch-Pagan test: p=0.000000


In [129]:
m2 = smf.ols('ceb ~ ' + ' + '.join(data.columns[1:]), data=data)
fitted = m2.fit(cov_type='HC1')
print(fitted.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.640
Model:                            OLS   Adj. R-squared:                  0.638
Method:                 Least Squares   F-statistic:                     335.3
Date:                Wed, 21 Apr 2021   Prob (F-statistic):               0.00
Time:                        18:20:36   Log-Likelihood:                -7757.4
No. Observations:                4348   AIC:                         1.555e+04
Df Residuals:                    4328   BIC:                         1.568e+04
Df Model:                          19                                         
Covariance Type:                  HC1                                         
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -0

In [130]:
m3 = smf.ols('ceb ~ age + educ + idlnchld + knowmeth + usemeth + agefm + heduc + urban + electric + bicycle + nevermarr '
             '+ idlnchld_noans + heduc_noans + usemeth_noans', data=data)
fitted = m3.fit()
print(fitted.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.640
Model:                            OLS   Adj. R-squared:                  0.638
Method:                 Least Squares   F-statistic:                     549.2
Date:                Wed, 21 Apr 2021   Prob (F-statistic):               0.00
Time:                        18:21:43   Log-Likelihood:                -7760.4
No. Observations:                4348   AIC:                         1.555e+04
Df Residuals:                    4333   BIC:                         1.565e+04
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -0.2824      0.429     -0.

In [131]:
print('Breusch-Pagan test: p=%f' % sms.het_breuschpagan(fitted.resid, fitted.model.exog)[1])

Breusch-Pagan test: p=0.000000


In [132]:
m4 = smf.ols('ceb ~ age + educ + idlnchld + knowmeth + usemeth + agefm + heduc + urban + electric + bicycle + nevermarr '
             '+ idlnchld_noans + heduc_noans + usemeth_noans', data=data)
fitted = m4.fit(cov_type='HC1')
print(fitted.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.640
Model:                            OLS   Adj. R-squared:                  0.638
Method:                 Least Squares   F-statistic:                     449.7
Date:                Wed, 21 Apr 2021   Prob (F-statistic):               0.00
Time:                        18:22:34   Log-Likelihood:                -7760.4
No. Observations:                4348   AIC:                         1.555e+04
Df Residuals:                    4333   BIC:                         1.565e+04
Df Model:                          14                                         
Covariance Type:                  HC1                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -0.2824      0.567     -0.

In [133]:
print('F=%f, p=%f, k1=%f' % m2.fit().compare_f_test(m4.fit()))

F=1.197092, p=0.307838, k1=5.000000


In [136]:
m5 = smf.ols('ceb ~ age + educ + idlnchld + knowmeth + agefm + heduc + urban + electric + bicycle + nevermarr '
             '+ idlnchld_noans + heduc_noans', data=data)
fitted = m5.fit(cov_type='HC1')
print(fitted.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.625
Model:                            OLS   Adj. R-squared:                  0.624
Method:                 Least Squares   F-statistic:                     391.1
Date:                Wed, 21 Apr 2021   Prob (F-statistic):               0.00
Time:                        18:26:51   Log-Likelihood:                -7844.3
No. Observations:                4348   AIC:                         1.571e+04
Df Residuals:                    4335   BIC:                         1.580e+04
Df Model:                          12                                         
Covariance Type:                  HC1                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.3436      0.531     -2.

In [135]:
print('F=%f, p=%.40f, k1=%f' % m4.fit().compare_f_test(m5.fit()))

F=85.265624, p=0.0000000000000000000000000000000000004783, k1=2.000000
